<h1 align=center><font size = 5>Week 3 Assignment for Capstone Project</font></h1>
<p style="font-size:20px"> Segmenting and Clustering Neighborhoods in Toronto </p>
<p style="color:blue"> This assignment aims to explore and cluster the neighborhoods in Toronto.</p>
<p> 1. creating a new Notebook for this assignment.</p>
<p> 2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe</p>
<p></p>
Import necessary Libraries

In [10]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import json # library to handle JSON files

from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


Loading dataset

In [14]:
source_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source_file = requests.get(source_url).text
bsoup = BeautifulSoup(source_file, 'html.parser')
Toronto =bsoup.find('table')

In [15]:
df_header = ['Postalcode','Borough','Neighborhood']
Toronto_tr = Toronto.tbody.find_all("tr")

In [54]:
temp = []
for tr in Toronto_tr:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned\n":
        for i, trow in enumerate(row):
            str = trow
            row[i] = str.replace("\n","")
        temp.append(row)
        
df = pd.DataFrame(temp, columns = df_header)

In [59]:
df

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


<p style="color:blue"> Part 3 : data cleansing and clustering. </p>
Join the neighborhood that exist in one postal code area 

In [56]:
df[df['Neighborhood']=='Not assigned']=df['Borough'].reset_index()

In [57]:
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Create a temporary dataframe by join the more than one neighborhood can exist in one postal code area

In [77]:
df_tmp=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df_tmp

Postalcode
M1B                                       Malvern, Rouge
M1C               Rouge Hill, Port Union, Highland Creek
M1E                    Guildwood, Morningside, West Hill
M1G                                               Woburn
M1H                                            Cedarbrae
                             ...                        
M9N                                               Weston
M9P                                            Westmount
M9R    Kingsview Village, St. Phillips, Martin Grove ...
M9V    South Steeles, Silverstone, Humbergate, Jamest...
M9W                  Northwest, West Humber - Clairville
Name: Neighborhood, Length: 103, dtype: object

Merge the temporary dataframe to df for the final output

In [78]:
df_tmp=df_tmp.reset_index(drop=False)
df_tmp.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)
df_merge = pd.merge(df, df_tmp, on='Postalcode')
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

df_merge.drop_duplicates(inplace=True)
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [87]:
df_merge.sort_values(by=['Postalcode'])
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [84]:
print("The shape of dataframe : ",df_merge.shape)

The shape of dataframe :  (103, 3)
